# STEP 1: Clonazione Darknet
Clonazione della repository personale (mi ha consentito di modificare il file "image.c" per croppare le immagini). All'interno sono presenti anche gli stream video (dir. "Video")

Repository di origine: https://github.com/AlexeyAB/darknet.git

In [ ]:
!git clone https://github.com/SpanoChristian/darknet.git

Creazione directory "dataset" che conterrà tutti i frame campionati dal video sorgente

In [ ]:
%cd ./darknet
%mkdir dataset/
%cd dataset/

# STEP 2: Campionamento frame da stream video

In [ ]:
%cd ../..
!wget http://hitis.altervista.org/CocktailParty.zip

In [ ]:
!unzip CocktailParty.zip
%cd ./darknet/dataset/

In [ ]:
import cv2
import random

def get_frame(stream):

  success = True
  count = 0

  video_sorgente = cv2.VideoCapture('../../CocktailParty/' + stream)
  fps = int(video_sorgente.get(cv2.CAP_PROP_FPS))

  if stream == "Cam_1.avi":
    camera = "c1"
  elif stream == "Cam_2.avi":
    camera = "c2"
  elif stream == "Cam_3.avi":
    camera = "c3"
  else:
    camera = "c4"

  while success:
    fps = int(video_sorgente.get(cv2.CAP_PROP_FPS))
    rnd_id = random.randint(0, 100000)
    success, image = video_sorgente.read()
    print('Lettura frame:', success)
    print('#Frame: ', fps)
    if count%(fps*20) == 0 and success==True:
         cv2.imwrite(str(rnd_id) + '_' + camera + '_%d.jpg'%count, image)
         print('Frame salvato con successo.')
    count += 1


**Campionamento frame dalle telecamere**

In [ ]:
get_frame("Cam_1.avi")
get_frame("Cam_2.avi")
get_frame("Cam_3.avi")
get_frame("Cam_4.avi")

Salvataggio del dataset scaricabile

In [ ]:
#!zip -r ../dataset.zip ../dataset/

# STEP 3: Verifica pre-requisiti e costruzione di Darknet
La costruzione consente di utilizzare il file eseguibile "darknet" che permette di avviare l'identificazione degli oggetti

In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd ..
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

In [ ]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

In [ ]:
# make darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)
!make

# STEP 4: Download pesi YOLOv4 pretrainati
Utilizziamo questi pesi pretrainati per eseguire YOLOv4 ed identificare le persone.

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

# STEP 5: Avvio dell'identificazione e crop delle immagini
Per ogni frame ricavato allo STEP 2) viene avviata l'identificazione delle persone presenti nell'immagine e, per ognuna di queste, viene salvata la relativa immagine croppata nella directory "bounding_box_test". Il dataset che si verrà a creare sarà successivamente dato in pasto all'algoritmo di person re-identification.

In [ ]:
# Creazione della directory dove si salveranno le persone croppate
%mkdir bounding_box_test/

In [ ]:
%%bash

cd ./dataset/
for f in *.jpg; do
    cd ..
    ./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights ./dataset/"$f"
    cd dataset/
done

In [ ]:
!zip -r ./bounding_box_test.zip ./bounding_box_test/